In [3]:
import pandas as pd
df = pd.read_csv("hf://datasets/azrai99/coursera-course-dataset/coursera_course_2024.csv")
import numpy as np

c:\Users\koush\python\ml\coursera_recommnedation\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df.drop(columns=['num_reviews','Level','Schedule','Satisfaction Rate','Unnamed: 0','enrolled'],inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6645 entries, 0 to 6644
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            6645 non-null   object
 1   rating           6645 non-null   object
 2   Instructor       6644 non-null   object
 3   Organization     6645 non-null   object
 4   Skills           6645 non-null   object
 5   Description      6635 non-null   object
 6   Modules/Courses  6635 non-null   object
 7   URL              6645 non-null   object
dtypes: object(8)
memory usage: 415.4+ KB


In [6]:
df.dropna(axis=0,inplace=True)

In [7]:
df.isnull().sum()

title              0
rating             0
Instructor         0
Organization       0
Skills             0
Description        0
Modules/Courses    0
URL                0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6634 entries, 0 to 6644
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            6634 non-null   object
 1   rating           6634 non-null   object
 2   Instructor       6634 non-null   object
 3   Organization     6634 non-null   object
 4   Skills           6634 non-null   object
 5   Description      6634 non-null   object
 6   Modules/Courses  6634 non-null   object
 7   URL              6634 non-null   object
dtypes: object(8)
memory usage: 466.5+ KB


In [9]:
df['Skills'][111]

"['Applying Generative AI in data projects', 'Anomaly detection with GenAI', 'Analyzing GenAI data security and privacy', 'Data augmentation with GenAI', 'Integrating GenAI into data workflows']"

In [10]:
from ast import literal_eval

In [11]:
df['Skills'] = df['Skills'].apply(literal_eval)

In [12]:
df['tags'] = df.apply(
    lambda row: list(filter(None, (
        str(row['title']) + ' ' +
        str(row['Description']) + ' ' +
        str(row['Instructor']) + ' ' +
        str(row['Organization'])
    ).split(" "))) + row['Skills'],
    axis=1
)

In [13]:
df['tags'] = df['tags'].apply(lambda x: " ".join(x))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
vectorizer = TfidfVectorizer(stop_words='english')
tag_vectors = vectorizer.fit_transform(df['tags'])

# 2. Compute cosine similarity matrix (NxN where N = number of courses)
similarity_matrix = cosine_similarity(tag_vectors)

In [16]:
len(similarity_matrix[0])

6634

In [17]:
df.shape

(6634, 9)

In [18]:
def get_top_5_similar(idx, similarity_matrix, df):
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_5 = [df.iloc[i[0]]['title'] for i in sorted_scores[1:6]]
    return top_5

In [19]:
df.reset_index(drop=True, inplace=True)
df['recommended_courses'] = df.index.to_series().apply(lambda idx: get_top_5_similar(idx, similarity_matrix, df))

In [20]:
df.to_pickle('courses_df.pkl')